In [12]:
import os

data = f'data/datasets/cam2.calibrate.rev.6.1/dataset2'

imagesPath = f'{data}/images/all'
labelsPath = f'{data}/labels/all'


In [13]:
from pathlib import Path

# создаём структуру папок

Path(f"{data}/images/all").mkdir(parents=True, exist_ok=True)
Path(f"{data}/images/train").mkdir(parents=True, exist_ok=True)
Path(f"{data}/images/test").mkdir(parents=True, exist_ok=True)
Path(f"{data}/images/val").mkdir(parents=True, exist_ok=True)

Path(f"{data}/labels/all").mkdir(parents=True, exist_ok=True)
Path(f"{data}/labels/train").mkdir(parents=True, exist_ok=True)
Path(f"{data}/labels/test").mkdir(parents=True, exist_ok=True)
Path(f"{data}/labels/val").mkdir(parents=True, exist_ok=True)


In [14]:
import os
import shutil

# переносим исходные данные аннотаций

for f in os.listdir(f'{data}/labels/train/frames'):
    shutil.move(f'{data}/labels/train/frames/{f}', f'{labelsPath}/{f}')
os.rmdir(f'{data}/labels/train/frames')

### Делим аннотации на части train/val/test:  70/20/10

In [15]:
import random

def moveRandom(src, dst):
    item = src[random.randint(0, len(src)-1)]
    src.remove(item)
    dst.append(item)

labels = os.listdir(labelsPath)
labels.sort()

random.seed()
test = []
[moveRandom(labels, test) for x in range(0, int(len(labels)/10))]

val = []
[moveRandom(labels, val) for x in range(0, int(len(labels)/10*2))]

train = labels

val.sort()
test.sort()
train.sort()

print('train:', len(labels))
print('val:', len(val))
print('test:', len(test))

train: 4169
val: 1042
test: 579


In [16]:
# Пишем файл-листы

out = open(f'{data}/val.txt', 'w')
for f in val:
    imgFile = f.replace(".txt", ".png")
    out.write(f'images/val/{imgFile}\n')
out.close()

out = open(f'{data}/test.txt', 'w')
for f in test:
    imgFile = f.replace(".txt", ".png")
    out.write(f'images/test/{imgFile}\n')
out.close()

out = open(f'{data}/train.txt', 'w')
for f in train:
    imgFile = f.replace(".txt", ".png")
    out.write(f'images/train/{imgFile}\n')
out.close()

In [17]:
# Move labels files

import shutil

[shutil.move(f'{labelsPath}/{f}', f'{data}/labels/train/{f}') for f in train]
[shutil.move(f'{labelsPath}/{f}', f'{data}/labels/val/{f}') for f in val]
[shutil.move(f'{labelsPath}/{f}', f'{data}/labels/test/{f}') for f in test]
print('Done')

Done


#### !!!! Copy all images to `images/all` folder !!!

In [18]:
# Move image files

for f in train:
    imgFile = f.replace(".txt", ".png")
    shutil.move(f'{imagesPath}/{imgFile}', f'{data}/images/train/{imgFile}')

for f in val:
    imgFile = f.replace(".txt", ".png")
    shutil.move(f'{imagesPath}/{imgFile}', f'{data}/images/val/{imgFile}')

for f in test:
    imgFile = f.replace(".txt", ".png")
    shutil.move(f'{imagesPath}/{imgFile}', f'{data}/images/test/{imgFile}')

In [19]:
# update data.yaml

f = open(f"{data}/data.yaml", 'r')
content = f.read()
f.close()
content = content.replace('train: train.txt', f'train: ./images/train')
content += f'val: ./images/val\n'
content += f'test: ./images/test\n'
print(content)

f = open(f"{data}/data.yaml", 'w')
f.write(content)
f.close()

kpt_shape:
- 5
- 3
names:
  0: level
path: .
train: ./images/train
val: ./images/val
test: ./images/test



### Клеим датасеты в один

In [23]:
sourceDatasets = [
    "data/datasets/cam2.calibrate.rev.6.1/dataset1",
    "data/datasets/cam2.calibrate.rev.6.1/dataset2"
]
prefixes = [
    "/kaggle/input/rev-6-1-p1-small/dataset1",
    "/kaggle/input/rev-6-1-p1-big/dataset2"
]

joinedDataset = "data/datasets/cam2.calibrate.rev.6.1/dataset.all"

f = open(f"{sourceDatasets[0]}/data.yaml", 'r')
content = f.read()
f.close()

content = content.replace('train: ./images/train', 'train: train.txt')
content = content.replace('val: ./images/val', 'val: val.txt')
content = content.replace('test: ./images/test', 'test: test.txt')
print(content)

f = open(f"{joinedDataset}/data.yaml", 'w')
f.write(content)
f.close()

for dataFile in ['train', 'val', 'test']:
    content = ''
    for s, p in zip(sourceDatasets, prefixes):
        f = open(f"{s}/{dataFile}.txt", 'r')
        data = f.read()
        f.close()
        data = data.replace('images/', f'{p}/images/')
        content += data
    f = open(f"{joinedDataset}/{dataFile}.txt", 'w')
    f.write(content)
    f.close()
        


kpt_shape:
- 5
- 3
names:
  0: level
path: .
train: train.txt
val: val.txt
test: test.txt

